<a href="https://colab.research.google.com/github/jolmcla/Machine-Learning-Operationalization/blob/master/Copia_de_regression_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [271]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [272]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

In [273]:
import os

print(os.getcwd())

/content


In [274]:
!tar xf /content/spark-3.2.0-bin-hadoop3.2.tgz

In [275]:
!pip install -q findspark

In [276]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [277]:
import findspark
findspark.init()


In [278]:
findspark.find()

'/content/spark-3.2.0-bin-hadoop3.2'

In [279]:
from pyspark.sql import SparkSession

spark= SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()

In [280]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
#df1 = pd.read_csv('/content/drive/MyDrive/heart.csv')

housing=spark.read.option("InferSchema",'true').csv("/content/drive/MyDrive/housing2.csv", header=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [281]:
housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [282]:
#columns of dataframe
housing.columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity']

In [283]:
#create sparksession object
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('lin_reg').getOrCreate()

In [284]:
#import Linear Regression from spark's MLlib
from pyspark.ml.regression import LinearRegression

In [285]:
#shape of dataset
print((housing.count(),len(housing.columns)))

(20640, 10)


In [286]:
#printSchema
housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [287]:
#view statistical measures of data 
housing.describe().show(5,False)

+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|summary|longitude          |latitude         |housing_median_age|total_rooms       |total_bedrooms    |population        |households       |median_income     |median_house_value|ocean_proximity|
+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|count  |20640              |20640            |20640             |20640             |20433             |20640             |20640            |20640             |20640             |20640          |
|mean   |-119.56970445736148|35.6318614341087 |28.639486434108527|2635.7630813953488|537.8705525375618 |1425.4767441860465|499.5396802325581|3.8706710029070246|206855.81690891474|null           |
|stddev |2.003531723

url = 'copied_raw_GH_link'
df1 = pd.read_csv(url)

In [288]:
#sneak into the dataset
housing.head(3)

[Row(longitude=-122.23, latitude=37.88, housing_median_age=41.0, total_rooms=880.0, total_bedrooms=129.0, population=322.0, households=126.0, median_income=8.3252, median_house_value=452600.0, ocean_proximity='NEAR BAY'),
 Row(longitude=-122.22, latitude=37.86, housing_median_age=21.0, total_rooms=7099.0, total_bedrooms=1106.0, population=2401.0, households=1138.0, median_income=8.3014, median_house_value=358500.0, ocean_proximity='NEAR BAY'),
 Row(longitude=-122.24, latitude=37.85, housing_median_age=52.0, total_rooms=1467.0, total_bedrooms=190.0, population=496.0, households=177.0, median_income=7.2574, median_house_value=352100.0, ocean_proximity='NEAR BAY')]

In [289]:
for i in housing.columns:
  print(i)
  housing.filter(housing[i].isNull()).show()


longitude
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+

latitude
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
+---------+---

In [290]:
housing_w_nulls=housing.filter(housing.total_bedrooms.isNotNull())
housing_w_nulls=housing_w_nulls.drop("ocean_proximity") 

In [291]:
#import corr function from pyspark functions
from pyspark.sql.functions import corr

# check for correlation
housing_w_nulls.select(corr('total_rooms','median_house_value')).show()

+-------------------------------------+
|corr(total_rooms, median_house_value)|
+-------------------------------------+
|                  0.13329413480832375|
+-------------------------------------+



In [292]:
housing_w_nulls.withColumn("longitude",(housing_w_nulls["longitude"]+200)).show(10,False)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|77.77    |37.88   |41.0              |880.0      |129.0         |322.0     |126.0     |8.3252       |452600.0          |
|77.78    |37.86   |21.0              |7099.0     |1106.0        |2401.0    |1138.0    |8.3014       |358500.0          |
|77.76    |37.85   |52.0              |1467.0     |190.0         |496.0     |177.0     |7.2574       |352100.0          |
|77.75    |37.85   |52.0              |1274.0     |235.0         |558.0     |219.0     |5.6431       |341300.0          |
|77.75    |37.85   |52.0              |1627.0     |280.0         |565.0     |259.0     |3.8462       |342200.0          |
|77.75    |37.85   |52.0

In [293]:
#printSchema
housing_w_nulls.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [294]:

#import vectorassembler to create dense vectors
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml import Pipeline



columns_to_scale = [x for x in housing_w_nulls.columns]
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_scale]
scalers = [MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_scale]
pipeline = Pipeline(stages=assemblers + scalers)
scalerModel = pipeline.fit(housing_w_nulls)
scaledData = scalerModel.transform(housing_w_nulls)


In [295]:
scaledData.columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'longitude_vec',
 'latitude_vec',
 'housing_median_age_vec',
 'total_rooms_vec',
 'total_bedrooms_vec',
 'population_vec',
 'households_vec',
 'median_income_vec',
 'median_house_value_vec',
 'longitude_scaled',
 'latitude_scaled',
 'housing_median_age_scaled',
 'total_rooms_scaled',
 'total_bedrooms_scaled',
 'population_scaled',
 'households_scaled',
 'median_income_scaled',
 'median_house_value_scaled']

In [296]:
scaledData=scaledData.select(['longitude_scaled','latitude_scaled','housing_median_age_scaled','total_rooms_scaled','total_bedrooms_scaled','population_scaled','households_scaled', 'median_house_value'])

In [297]:
#import vectorassembler to create dense vectors


In [298]:
#create the vector assembler 
vec_assmebler=VectorAssembler(inputCols=['longitude_scaled','latitude_scaled','housing_median_age_scaled','total_rooms_scaled','total_bedrooms_scaled','population_scaled','households_scaled'],outputCol='features')

In [299]:
#transform the values
features_df=vec_assmebler.transform(scaledData)

In [300]:
#validate the presence of dense vectors 
features_df.printSchema()

root
 |-- longitude_scaled: vector (nullable = true)
 |-- latitude_scaled: vector (nullable = true)
 |-- housing_median_age_scaled: vector (nullable = true)
 |-- total_rooms_scaled: vector (nullable = true)
 |-- total_bedrooms_scaled: vector (nullable = true)
 |-- population_scaled: vector (nullable = true)
 |-- households_scaled: vector (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- features: vector (nullable = true)



In [301]:
#view the details of dense vector
features_df.select('features').show(5,False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                    |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|[0.21115537848605498,0.5674814027630182,0.7843137254901961,0.022330739101683705,0.019863438857852266,0.008940833543541018,0.020555829633284]|
|[0.21215139442231046,0.5653560042507968,0.39215686274509803,0.180502568798006,0.17147734326505276,0.06721040387903249,0.18697582634435125]  |
|[0.21015936254980092,0.5642933049946866,1.0,0.03726028790884582,0.029329608938547486,0.013817651840017937,0.028942608123663872]             |
|[0.2091633466135454,0.5642933049946866,1.0,0.03235159468945521,0.03631284916201117,0.015555368704279827,0.0358493668804473]                 |

In [304]:
#create data containing input features and output column
model_df=features_df.select('features','median_house_value')


In [305]:
model_df.show(5,False)

+--------------------------------------------------------------------------------------------------------------------------------------------+------------------+
|features                                                                                                                                    |median_house_value|
+--------------------------------------------------------------------------------------------------------------------------------------------+------------------+
|[0.21115537848605498,0.5674814027630182,0.7843137254901961,0.022330739101683705,0.019863438857852266,0.008940833543541018,0.020555829633284]|452600.0          |
|[0.21215139442231046,0.5653560042507968,0.39215686274509803,0.180502568798006,0.17147734326505276,0.06721040387903249,0.18697582634435125]  |358500.0          |
|[0.21015936254980092,0.5642933049946866,1.0,0.03726028790884582,0.029329608938547486,0.013817651840017937,0.028942608123663872]             |352100.0          |
|[0.2091633466135454,0.56429

In [306]:
#size of model df
print((model_df.count(), len(model_df.columns)))

(20433, 2)


In [307]:
#split the data into 70/30 ratio for train test purpose
train_df,test_df=model_df.randomSplit([0.7,0.3])



In [308]:
print((train_df.count(), len(train_df.columns)))

(14313, 2)


In [309]:
print((test_df.count(), len(test_df.columns)))

(6120, 2)


In [310]:
train_df.describe().show()

+-------+------------------+
|summary|median_house_value|
+-------+------------------+
|  count|             14313|
|   mean|207522.14734856423|
| stddev|115873.43907820669|
|    min|           14999.0|
|    max|          500001.0|
+-------+------------------+



In [311]:
#Build Linear Regression model 
lin_Reg=LinearRegression(labelCol='median_house_value')

In [312]:
#fit the linear regression model on training data set 
lr_model=lin_Reg.fit(train_df)

In [313]:
lr_model.intercept

751413.704461604

In [314]:
print(lr_model.coefficients)

[-705545.4833313901,-673208.2184071423,20420.42506055536,1879877.7614370377,-1094330.0020162405,-2465390.9641257212,800033.7883019286]


In [315]:
training_predictions=lr_model.evaluate(train_df)

In [316]:
training_predictions.meanSquaredError

8275420101.514902

In [317]:
training_predictions.r2

0.3836142356373503

In [319]:
#make predictions on test data 
test_results=lr_model.evaluate(test_df)

In [320]:
#view the residual errors based on predictions 
test_results.residuals.show(10)

/content/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------------------+
|          residuals|
+-------------------+
|-114021.24787405948|
| -123813.9040577513|
|-12852.731273540063|
| -92219.12272275297|
|-110959.37325960398|
| -90190.86963967653|
| -96695.75417783775|
| -98313.56446928647|
| -67220.13948030479|
| -87709.28732038522|
+-------------------+
only showing top 10 rows



In [321]:
#coefficient of determination value for model
test_results.r2

0.37885343274679995

In [322]:
test_results.rootMeanSquaredError

90154.49986887307

In [323]:
test_results.meanSquaredError

8127833846.606633